In [ ]:



from airtest.core.api import *
from poco.drivers.android.uiautomation import AndroidUiautomationPoco




poco = AndroidUiautomationPoco()
poco.device.wake()

In [ ]:
import re
res = poco("android.widget.ScrollView").child("android.view.View")
print(res)
views = [i for i in res]
print(views[1].get_text())
txt = views[1].get_text().replace(' ', ' ')

print(txt)
res = re.findall('(?<=[^\d])\d+(?=人想要)', txt)
print(res)
res = re.findall('￥\s\d+(\.\d+)?\s(\d+人想要)*', txt.replace('\n', ' '))
print(res)

In [ ]:
if(poco(text="向右滑动验证").exists):
    print('jjj')
    pos = poco("nc_1_n1t").attr('pos')
    mid = poco(text="向右滑动验证").attr('pos')
    print(pos)
    print([mid])
    w = mid[0] - pos[0]
    mid[0] = mid[0] + w
    print(mid)
    x1 = int(pos[0] * 1080)
    y1 = int(pos[1] * 1920)
    x2 = int(mid[0] * 1080)
    y2 = int(mid[1] * 1920)
    adb = poco.adb_client
    print(adb.get_display_info())
    swipe([x1, y1], [x2, y2], duration = 2, steps = 30)
 

    

In [ ]:
if(poco(text="刷新").exists()):
    poco(text="刷新").click()

In [ ]:
adb = poco.adb_client
adb.shell('am broadcast -a clipper.get')

In [ ]:
img = adb.snapshot()

In [ ]:
print(img)

In [ ]:
import base64
import matplotlib.image as image
import matplotlib.pyplot as plt
from PIL import Image
import io
import datetime
import uuid
import random
import os

########################################
# pos:中心点位置(x,y)
# size: (width, height)
def getFishSnapShot(pos, size):

    res = poco.snapshot()
    img_data = base64.b64decode(res[0])

    ff = io.BytesIO(img_data)
    img = image.imread(ff, res[1])
    # img2 = Image.open(ff)
    # img2.show()
    shape = img.shape
    screen_h = shape[0]
    screen_w = shape[1]

    pos_x = int(pos[0]*screen_w)
    pos_y = int(pos[1]*screen_h)

    item_w = int(screen_w * size[0])
    item_h = int(screen_h * size[1])
    item_begin_x = pos_x - item_w//2 -1
    item_begin_y = pos_y - item_h//2 - 1
    item_end_x = item_begin_x + item_w + 2
    item_end_y = item_begin_y + item_h + 2
    img2 = img[item_begin_y:item_end_y, item_begin_x:item_end_x, :]
    plt.axis('off')
    #plt.imshow(img2)
    #img_name = str(uuid.uuid1()) + ".png"
    img_name = "Fish-" + datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S-")+ str(random.randint(1000000,10000000)) + ".png"
    img_dir = 'img'
    if(not os.path.exists(img_dir)):
        os.mkdir(img_dir)
    Image.fromarray(img2).save( img_dir + '/' + img_name)
    # with open('1.'+res[1], 'wb') as f:
    #     f.write(img_data)
    #     f.close()
    return img_name


In [ ]:
views = poco("android.widget.ScrollView").child("android.view.View")
for i in views:
    print('###################')
    txt = i.get_text()
    if(txt):
        pos = i.attr('pos')
        size = i.attr('size')
        print(pos)
        print(size)
        getFishSnapShot(pos, size)
    #print(i.get_text())

In [ ]:
datetime.datetime.now().strftime("%Y-%m-%d %H%M%S ")+ str(random.randint(1000000,10000000))

In [ ]:
poco("com.taobao.idlefish:id/tx_id").click()

In [ ]:
poco("android.widget.EditText").setattr('text', 'ddd')
# text('ddddd')

In [ ]:
res = adb.shell('pm list packages')
ary = res.split('\r\n')
ary.sort()
for i in ary:
    print(i)


In [ ]:
adb.shell('pm path com.taobao.idlefish')

In [ ]:
adb.pull('/data/app/com.taobao.idlefish-1/base.apk', './idle_fish.apk')

In [ ]:
import pymongo
from pymongo.errors import DuplicateKeyError

mongoClient = pymongo.MongoClient('mongodb://localhost:27017')

mydb = mongoClient['idle_fish']
fish_list = mydb['items']
block_nickname = mydb['block_nickname']

# fish_list.create_index([("url_id", pymongo.DESCENDING)], unique = True)
# item = {'url_id':1234, 'title':"iphone8"}
# x = fish_list.insert_one(item)

# res = fish_list.find_one({'url_id':'6364531625454'})
# print(res)

# user_Info = mydb['user_info']


# # 排除搜索
# res = block_nickname.find({}, {"_id":0,'nickname': 1})
# block_nickname_list = [i['nickname'] for i in res]
# print(block_nickname_list)

# filter = {'nickname':{'$nin':block_nickname_list}}
# res = fish_list.find( filter,{'_id':0, 'nickname':1}).sort([('img_name',-1)]).limit(20)
# print([i for i in res])

        

## 添加block_nickname

In [ ]:
import pymongo
from pymongo.errors import DuplicateKeyError

mongoClient = pymongo.MongoClient('mongodb://localhost:27017')

mydb = mongoClient['idle_fish']
fish_list = mydb['items']
block_nickname = mydb['block_nickname']


# 若某个nickname名下fish超过给定值，则很可能非个人卖家，加入屏蔽名单
def addBlocknickname(nickname):
    try:
        block_nickname.insert_one({'nickname': nickname})
        print("add %s to block_nickname!"% nickname)
    except DuplicateKeyError as err:
        #print('DuplicateKey %s'% nickname)
        pass
res = fish_list.aggregate([{"$group": {"_id":"$nickname", "fish_num":{"$sum":1}}}])
for i in res:
    #print(i)
    ll = ['通讯', '数码', '店', '手机', '批发', '二手', '工作室', '回收', '出售', '优品', 'iphone', '交易']
    for kw in ll:
        if(i['_id'].find(kw) > -1):
            addBlocknickname(i['_id'])
            break
    if(i['fish_num'] > 3):
        addBlocknickname(i['_id'])
        

## 图片链接

In [ ]:
# import pymongo
# from pymongo.errors import DuplicateKeyError

# mongoClient = pymongo.MongoClient('mongodb://localhost:27017')

# mydb = mongoClient['idle_fish']
# fish_list = mydb['fish']
# block_nickname = mydb['block_nickname']
# db_items = mydb['items']

# res = fish_list.find({}).limit(2)
# def replaceCol(a, src, des):
#     a[des] = a[src]
#     del a[src]
# for i in res:
#     i['item_id'] = int(i['url_id'])
#     del i['url_id']
#     replaceCol(i, 'wanted', 'want')
#     #replaceCol(i, 'url_id', 'item_id')
#     replaceCol(i, 'user_id', 'unique_name')
#     replaceCol(i, 'location', 'area')
#     replaceCol(i, 'detail', 'desc')

#     i['pic_url'] = 'img/' + i['img_name']
#     del i['img_name']
#     db_items.insert_one(i)
#     print(i)


In [13]:
import re

re.findall('果|iphone','4IdPhone'.lower())

[]

In [15]:
from subprocess import call

cmd = 'display notification \"' + \
    "Notificaton memo\n\ndddd\nddd\nd\nd\nd" + '\" with title \"Titile\"'
call(["osascript", "-e", cmd])



0

In [16]:
import json

a = {'dd':45, 'bb':"sss"}
json.dumps(a)

'{"dd": 45, "bb": "sss"}'